Using the functions and loops created in the webscraper notebook.  Here I'm going to loop through one webpage and get all 20 tables into one dataframe

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
from selectorlib import Extractor
import bs4
import requests
import json
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('ignore-certificate-errors')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)

In [ ]:
url = "https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/saison_id/2020"

driver.get(url)
html = driver.execute_script('return document.body.innerHTML;')
soup = bs4.BeautifulSoup(html,'lxml')